In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sympy import sieve
import time

In [2]:
nodes = pd.read_csv('./problem/cities.csv')
nodes = nodes.iloc[:,1:].values
n = len(nodes)
prime_ = sieve.primerange(0,n)
prime = list(prime_)

In [3]:
def score(solution,offset=0,raw=False):
    s = nodes[solution[:-1]]
    e = nodes[solution[1:]]
    out = distance(s,e)
    n = len(s)
    if raw:
        return out
    isten = (np.arange(n)+1+offset)%10==0
    notprime = ~np.isin(solution[:-1],prime)
    isextra = (isten&notprime)*0.1
    out += out*isextra
    return out

def distance(coor1,coor2):
    return np.sqrt(((coor1-coor2)**2).sum(axis=-1))


In [4]:
solution = pd.read_csv('./solution/base.csv').values[:,0]
score(solution).sum()

1516912.3751145517

In [44]:
r_solution = solution[-2:0:-1]
reverse_dist = []
for offset in range(10):
    reverse_dist.append(np.cumsum(score(r_solution,offset)[::-1])[::-1])
reverse_dist = np.array(reverse_dist)
loop = reverse_dist.shape[1]

In [48]:
dist = np.cumsum(score(solution))

In [52]:
asd = []
start = time.time()
for left in range(1900):
    node_before_a, node_a = solution[left:left+2]
    offset = (left+1)%10
    middle = np.zeros(shape=(n-2-left))
    for o in range(10):
        order = (offset-o)%10
        using_reverse = reverse_dist[order][:n-2-left]
        middle[o::10] = using_reverse[o::10]
    reverse_dist[:,:loop-left-1] -= reverse_dist[:,loop-left-1:loop-left]
    asd.append(middle[0])
    
    old_dist = dist[left+2:]
    
    dist[left+1:] -= dist[left]
print(time.time()-start)       

6.107999801635742


In [51]:
dist[:10]

array([ 8.71387618,  7.24548268,  5.9463006 ,  7.49827218,  8.53004849,
        8.84179965, 17.95567891, 26.50275793, 33.26565234, 39.7231002 ])

In [49]:
dist[:10]

array([ 8.71387618, 15.95935886, 21.90565946, 29.40393164, 37.93398013,
       46.77577979, 55.88965904, 64.43673807, 71.19963247, 77.65708034])

# UTILITY

In [20]:
#write result
pd.DataFrame({'Path':solution}).to_csv(path+'solution/submit2.csv',index=False)

In [21]:
#make .tsp problem
fname = path + 'problem/santa.tsp'
with open(fname, 'w') as f:
    f.write('NAME : santa\n')
    f.write('COMMENT : santa\n')
    f.write('TYPE : TSP\n')
    f.write('DIMENSION : %d\n' % n)
    f.write('EDGE_WEIGHT_TYPE : EUC_2D\n')
    f.write('NODE_COORD_SECTION\n')
    for idx,node in enumerate(nodes):
        x,y=node*100000
        f.write('%s %s %s\n' % (idx+1,x,y))
    f.write('EOF\n')

In [24]:
#read neos html
fname = path + 'solution/neos2.html'
l = 1
sol = []
with open(fname, 'r') as f:
    while l< 207274:
        line = f.readline()
        if l>=9505:
            sol.append(int(line.split(' ')[0]))
        l += 1
sol.append(0)
solution = np.array(sol)

# node swap

In [14]:
start = time.time()

#every iter
coor = nodes[solution]
dist = score(solution)
neighbor = pd.DataFrame({'d':dist}).d.rolling(2,center=True).sum().values
oldnext = pd.DataFrame({'d':dist}).d.rolling(3,center=True).sum().values
notprime = ~np.isin(solution[:-1], prime)
isten = (np.arange(n)+1)%10==0
isextra = isten&notprime
better = 0

time_a = time.time()

cur = np.append([0],distance(coor[0],coor))
nex = np.append([0],distance(coor[1],coor))

swap = {}
for left in range(1,n-1):
#every left
    pre_coor, left_coor, next_coor = coor[left-1:left+2]
    adj_dist = distance(left_coor, next_coor)
    
    FROM_FLAG = (left+1)%10 == 0
    TO_FLAG = left%10 == 0
    LEFT_FLAG = notprime[left]
    LEFT_pre_FLAG = notprime[left-1]
    LEFT_nex_FLAG = notprime[left+1]
    
    ##old cost
    left_nb = neighbor[left]
    old_cost = neighbor[left:] + left_nb
    old_cost[0] = 0
    old_cost[1] = oldnext[left]

    ##new cost
    pre = cur[1:]
    
    cur = nex[1:]
    
    nex = distance(next_coor,coor[left-1:])

    cur_1 = cur[:-2]
    cur_2 = cur[2:]
    pre_ = pre[1:-1]
    nex_ = nex[1:-1]
    
    if TO_FLAG & LEFT_pre_FLAG:
        pre_ = pre_ + pre_*0.1
    if FROM_FLAG:
        nex_ = nex_ + nex_*notprime[left:]*0.1
        
    cur_1 = cur_1 + cur_1*isextra[left-1:-1]*0.1
    
    if LEFT_FLAG:
        cur_2 = cur_2 + cur_2*isten[left:]*0.1
    
    if FROM_FLAG & LEFT_nex_FLAG:
        adj_dist *= 1.1
    new_cost = pre_ + nex_ + cur_1 + cur_2
    
    
    new_cost[0] = 0
    new_cost[1] += adj_dist
    
    save = old_cost - new_cost
    good = np.where(save>0)[0]
    
    if len(good)>0:
        good_save = save[good]
        swap[left] = [good+left,good_save]
    
    if left%10000 == 0:
        time_b = time.time()
        print(left,time_b-time_a)
        time_a = time_b
print(time.time()-start)
print(len(swap))
###############################################
bk = set()
candidate = []
for left in swap:
    right,improve = swap[left]
    for i in range(len(right)):
        candidate.append((improve[i],right[i],left))
        
candidate = sorted(candidate)[::-1]
l=[]
r=[]
for improve,left,right in candidate:
    fill = [left-1,left,left+1,right-1,right,right+1]
    fill = [not (x in bk) for x in fill]
    if all(fill):
        l.append(left)
        r.append(right)
        bk.add(left-1)
        bk.add(left)
        bk.add(left+1)
        bk.add(right-1)
        bk.add(right)
        bk.add(right+1)

buf = solution[l]
solution[l] = solution[r]
solution[r] = buf

score(solution).sum()

10000 126.58199977874756
20000 121.46800017356873
30000 112.0699999332428
40000 105.73300004005432
50000 98.03699994087219
60000 93.94600009918213
70000 85.03600001335144
80000 58.355000019073486
90000 50.651999950408936
100000 45.80800008773804
110000 40.49499988555908
120000 35.301000118255615
130000 31.38599991798401
140000 24.236000061035156
150000 19.155999898910522
160000 15.098999977111816
170000 11.743000030517578
180000 8.926999807357788
190000 4.612000226974487
1090.026999950409
0
